In [108]:
import yfinance as yf
import time

import matplotlib.pyplot as plt
import math
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import timedelta


from ta.momentum import roc
from ta.momentum import RSIIndicator,rsi
from ta.trend import wma_indicator, ema_indicator
from ta.utils import dropna

import warnings
warnings.filterwarnings('ignore')

# Trading Strategy

Combination of a momentum indicator **Coppock Curve** with a mean-reversion indictor **RSI**. 

Dual-indicator approach that leverages the strengths of both the Coppock Curve and the RSI to maximize trading opportunities. This strategy is designed to harness the Coppock Curve's ability to identify long-term trend reversals and the RSI's sensitivity to short-term price movements. This combination allows traders to align themselves with the overall market momentum while pinpointing optimal entry points for trades. Ultimately, offering a more holistic view of the market and potentially filter out false signals.


## Signals
We will use Coppock Curve and RSI to each generate a value. 1 or -1. 

### Coppock curve [1,-1]
1 is generated when Coppock Curve crosses above zero. This indicates that the momentum has shifted from negative (bearish) to positive (bullish), suggesting that it might be a good time to buy.
-1 is generated when the Coppock Curve crosses below zero. This suggests that the momentum has shifted from positive (bullish) to negative (bearish), which might indicate that it might be a good time to short.

### RSI [1,-1]
40/70 We will use the values 40 to indicate oversold and 70 to indicate overbought.
1 is generated when RSI crossed above 40 from below or if RSI >50
-1 is generated when RSI crossed below 70 from above or if RSI <50

Combining the values generated by Coppock Curve and RSI
We will be able to generate the absolute value (strength of signal) denoting the confidence and direction of the signal

- **Strong Buy Signal**: (1,1) 2
- **Weak Buy Signal**: (1,-1) 0
- **Strong Sell Signal**: (-1,1) 0
- **Weak Sell Signal**: (-1,-1) -2

We will then convert the value generated to get the weights relative to the Portfolio

Legend :
2 = 3 weight  
0 = 2 weight
-2 = 1 weight

We will then use the weights given to the stock to generate the capital allocated for each stock.
Capital of Stock x = Weights of Stock x / Sum of weights of all stock

## Variations of Coppock Curve

1. **Coppock Curve with standard sets of 2 periods**: The standard Coppock Curve uses a 14-day period for the long Rate of Change (RoC) and an 11-day period for the short RoC. An alternative configuration might be to use a 21-day period for the long RoC and a 5-day period for the short RoC.

2. **Coppock Curve with different smoothing constants**: The standard Coppock Curve uses a 10-period Weighted Moving Average (WMA) for smoothing.
Alternative configurations might use a different period for the WMA.

3. The standard Coppock Curve uses a **Weighted Moving Average (WMA)** for smoothing.

In [158]:
# Strategy Logic
# Coppock Curve and RSI strategy

def get_cc_wma(df,roc1_n,roc2_n,wma_lookback):
    cc_wma = wma_indicator((roc(close=df['Close_Price'], window=roc1_n,fillna= False)) + (roc(close=df['Close_Price'], window=roc2_n,fillna= False)), window=wma_lookback,fillna= False)
    return cc_wma

def get_rsi(df,window):
    rsi_indicator = rsi(close=df['Close_Price'], window=window,fillna= False)
    return rsi_indicator

def CC_RSI_strategy(df,num):
    wma = 10
    for var in range(1,num+1):
        df[f'cc_{var}'] = get_cc_wma(df, 14, 11, wma)
        wma += 40
    df['rsi_14'] = get_rsi(df, 14)
    df = df.dropna()

    for var in range(1,num+1):
        cc_crossed_above_zero = False
        rsi_crossed_40_below = False
        rsi_crossed_70_above = False
        val = [1]
        for day in range(1,len(df)):
            # Generate strat
            cc_current = df.iloc[day][f'cc_{var}']
            cc_prev = df.iloc[day-1][f'cc_{var}']
            rsi_current = df.iloc[day]['rsi_14']
            rsi_prev = df.iloc[day-1]['rsi_14']
            if cc_crossed_above_zero:
                # check rsi
                if rsi_crossed_40_below or rsi_current > 50:
                    val.append(3)
                elif rsi_crossed_70_above or rsi_current < 50:
                    val.append(2)
                else:
                    val.append(1)

                if cc_current <= 0:
                    cc_crossed_above_zero = False
                if rsi_current < 40:
                    rsi_crossed_40_below = False
                if rsi_current > 70:
                    rsi_crossed_70_above = False
                # continue
            else:
                if rsi_crossed_40_below or rsi_current > 50:
                    val.append(2)
                elif rsi_crossed_70_above or rsi_current < 50:
                    val.append(1)
                else:
                    val.append(3)  
            # Check if CC has crossed above zero
            if cc_current > 0 and cc_prev <= 0:
                cc_crossed_above_zero = True
            else:
                cc_crossed_above_zero = False

            # Check if RSI has crossed 40 from below
            if rsi_current > 40 and rsi_prev <= 40:
                rsi_crossed_40_below = True
            else:
                rsi_crossed_40_below = False

            # Check if RSI has crossed 70 from above
            if rsi_current < 70 and rsi_prev >= 70:
                rsi_crossed_70_above= True
            else:
                rsi_crossed_70_above = False

        df[f'cc_{var}_val'] = val
        df[f'cc_{var}_qty'] = None
    return df
    

In [159]:
class Portfolio():
    def __init__(self, stocks, capital, variations):
        self.stocks = stocks
        self.dataframe = pd.DataFrame()  # load this dataframe sequentially everyday
        self.capital = capital  # this capital will change depending on trades
        self.variations = variations
        self.initialise_capital()
    
    # TODO: generate overall performance metrics on the Portfolio_{var}_value
    
    # Call add_benchmark to add benchmark to self.dataframe

    # 1)Need add functions to calc the returns, log returns
    # 2)Performance metrics:- compute_sharpe_ratio,compute_drawdown,compute_calmar_ratio
    # 3)Alpha, Beta of the Portfolio against benchmark
    # 4)Strength of signal against the price of the stock 

    def overall_performance_metrics(self,):
        pass

    # TODO: Year on year performance metrics to evaluate the performance of the strategy
    # 1) yearly evaluation graphs
    def yearly_performance_metrics(self,):
        pass

    def add_benchmark(self,benchmark_symbol):
        start_date = self.dataframe.index[0]
        end_date = self.dataframe.index[-1] + timedelta(days=1)
        benchmark_data = yf.download(benchmark_symbol, start=start_date, end=end_date)
        print(benchmark_data.shape)
        print(self.dataframe.shape)
        if benchmark_data.shape[0] == self.dataframe.shape[0]:
            print('Match')
            self.dataframe[f'benchmark_{benchmark_symbol}_close'] = benchmark_data['Close']

    def initialise_capital(self):
        '''
        self.initial_1_capital (initial capital of the day)
        self.capital_1 (live capital update)
        '''

        for i in range(1,self.variations+1):
            setattr(self, f'initial_{i}_capital', self.capital)
            setattr(self, f'capital_{i}', self.capital)
            
    def calculate_weights(self, date, var):
        total_weight = 0
        weights = {}

        for stock_name, stock in self.stocks.items():
            weight = stock.data.loc[date][f'cc_{var}_val']
            weights[stock_name] = weight
            total_weight += weight
        # Normalize weights
        for stock_name in weights:
            if total_weight != 0:
                weights[stock_name] /= total_weight
        return weights
    
    def generate_weightage(self, dates):
        '''
        dates: list of dates from start date to end date
        '''
        # Generate a dictionary of indiv stock and weights for each day
        # Normalise the weights to 100%
        for date in dates:
            for var in range(1, self.variations + 1):
                weights = self.calculate_weights(date,var)
                # create a column in each stock.data dataframe
                for stock_name, stock in self.stocks.items():
                    stock.data.loc[date, f'cc_{var}_weights'] = weights[stock_name]

    def run_simulation(self, dates):
        for date in range(len(dates)):
            today = dates[date]
            tmr = dates[date + 1] if date != (len(dates)-1) else None
            past = dates[date - 1] if date > 0 else None

            # Close all positions and get capital back
            self.close_positions(today,past)

            # Update portfolio after closing positions
            for var in range(1,self.variations + 1):
                column_name = f'Portfolio_{var}_value'
                initial_capital_attr = f'initial_{var}_capital'
                initial_capital_value = getattr(self, initial_capital_attr)
                self.dataframe.at[today, column_name] = initial_capital_value

            if tmr == None:
                break

            for var in range(1, self.variations + 1):
                # For each variation
                for stock_name, stock in self.stocks.items():
                    # For each stock
                    # Initial capital - Used for calculating purchasing power, only update after all stocks in portfolio expanded
                    # Capital - Used for getting live update on capital left, update initial capital.
                    initial_capital = getattr(self,  f'initial_{var}_capital')
                    purchasing_power = initial_capital * stock.data.loc[today, f'cc_{var}_weights']
                    close_price = stock.data.loc[today, 'Close_Price']
                    qty = math.floor(purchasing_power / close_price)
                    
                    live_capital = getattr(self, f'capital_{var}')
                    stock_capital = close_price * qty
                    live_capital -= stock_capital
                    setattr(self, f'capital_{var}', live_capital)
                    
                    stock.data.loc[tmr, f'cc_{var}_qty'] = qty

                    # TODO: (Question: should we add the stock capital to portfolio dataframe)
                    # If yes the data is bellow
                    # self.dataframe.at[today, f'Portfolio_{var}_{stock_name}'] = stock_capital
                    
                    print(f'Bought {qty} shares of {stock_name} - Total Capital Left : {live_capital}')
                setattr(self, f'initial_{var}_capital', live_capital)

    def close_positions(self, today_date, prev_date):
        # check if there are any positions
        # check the qty of stocks
        for var in range(1, self.variations + 1):
            for stock_name, stock in self.stocks.items():
                if (prev_date == None):
                    print(f'Day 0')
                    continue
                else:
                    # Sell and add back to portfolio money
                    # Sell at open price and add back money to portfolio
                    open_price = stock.data.loc[today_date, 'Open_Price']
                    qty = stock.data.loc[today_date, f'cc_{var}_qty']
                    live_capital = getattr(self, f'capital_{var}')
                    live_capital += (open_price * qty)
                    setattr(self, f'capital_{var}', live_capital)
                    setattr(self, f'initial_{var}_capital', live_capital)
                    # print(f'Sold {qty} shares of {stock_name} - Total Capital Left : {live_capital}')


In [160]:
class Stock():

  def __init__(self, symbol, start, end, interval, transcation_cost, strategy, verbose = True):
    self.symbol = symbol
    self.start = start
    self.end = end
    self.interval = interval
    self.transaction_cost = transcation_cost # the transaction cost for trading
    self.quantity = 0 # quantities to buy/sell
    self.position = 0 # the trades in progress, long or short
    self.trades = 0 # Number of trades
    self.verbose = verbose # if you want to see detailed output (logs)
    self.strategy = strategy # define the strategy

    self.prepare_data() # prepares the data

  def prepare_data(self):
    
    # since we are building a common class for all types of strategy, we will not calcualte the moving averages now.
    # we will calculate the returns though.
    # Since most strategies utilise close prices we are only factoring close price. However, you can alter acoordingly.

    stock_data = yf.Ticker(self.symbol)
    hist_stock = stock_data.history(start = self.start, end = self.end, interval = self.interval)
    bt_data = pd.DataFrame()
    bt_data["Close_Price"] = hist_stock["Close"]
    bt_data["Open_Price"] = hist_stock["Open"]
    bt_data["Return"] = np.log(bt_data["Close_Price"] / bt_data["Close_Price"].shift(1))
    bt_data = bt_data.dropna()
    bt_data.index = bt_data.index.date
    self.data = bt_data
    
  def close_graph(self):
    plt.figure(figsize=(15, 5))
    plt.plot(self.data["Close_Price"] ,color='black', label='Price', linestyle='dashed')
    plt.xlabel("Days")
    plt.ylabel("Price")
    plt.title("Close Prices of {}".format(self.symbol))
    plt.legend()
    plt.grid()
    plt.show()

  def return_date_price(self, bar):
    '''
    bar: is a unit of data
    '''
    # A bar is a unit of data at a given time, depends on the interval you choose, it provides you OHLCV and time info
    # Since we have modeled close prices, we will get the price and date

    date = str(self.data.index[bar])[:10] #First 10 contains the date elements, rest is time
    price = self.data.Close_Price.iloc[bar]
    return date, price

  def load_strategy(self,num):
    '''
    num: number of variations to create
    '''

    # Load the strategy to generate key columns

    self.data = self.strategy(self.data,num)

In [161]:
from matplotlib.ticker import FormatStrFormatter

class Backtester():
    
    def __init__(self, portfolio, variations):
        self.portfolio = portfolio
        self.variations = variations
        
    def check_unique_signal_count(self):
        # Validate and check the different strength across stocks
        for stock_symbol, stock in self.portfolio.stocks.items():
            print(stock_symbol)
            for i in range(1,self.variations+1):
                print(stock.data[f'cc_{i}_val'].value_counts())
            print('\n')

    def validate_stock_data(self):
        # Validate and check if stock data consistent
        data_shapes = {stock.data.shape for stock in self.portfolio.stocks.values()}
        return len(data_shapes) == 1, data_shapes.pop() if data_shapes else None

    def validate_start_dates(self):
        start_dates = set()
        # Validate and get the start date of the backtest
        for stock_symbol, stock in self.portfolio.stocks.items():
            start_dates.add(stock.data.index[0])
        return len(start_dates) == 1, start_dates.pop() if start_dates else None
        
    def validate_end_dates(self):
        end_dates = set()
        # Validate and get the end date of the backtest
        for stock_symbol, stock in self.portfolio.stocks.items():
            end_dates.add(stock.data.index[-1])
        return len(end_dates) == 1, end_dates.pop() if end_dates else None

    def verify(self):
        flag_stocks, shape = self.validate_stock_data()
        flag_start_date, start_date= self.validate_start_dates()
        flag_end_date, end_date = self.validate_end_dates()
        verify = flag_stocks and flag_start_date and flag_end_date
        self.start_bt_date = start_date
        self.end_bt_date = end_date
        return verify

    def prepare_strategy(self):
        # Code to start backtester
        stocks = self.portfolio.stocks
        for stock_name, stock in self.portfolio.stocks.items():
            stock.load_strategy(self.variations)
        if self.verify():
            first_stock_symbol = next(iter(stocks))
            dates = list(self.portfolio.stocks[first_stock_symbol].data.index)
            self.bt_dates = dates
            self.portfolio.generate_weightage(dates)
        else:
            return Exception
    def run(self):
        # Populate portfolio dataframe 
        self.portfolio.run_simulation(self.bt_dates)

In [162]:
# create stock -> create portfolio -> create backtester 

stock_symbols = ['NVDA','AMZN','AAPL','PG','GOOGL']

stocks = [ Stock(
    symbol=symbol,
    start='2010-01-01',
    end='2016-12-31', 
    interval='1d', 
    transcation_cost=0, 
    strategy=CC_RSI_strategy, 
    verbose=True) 
    for symbol in stock_symbols
    ]

CC_RSI_portfolio = Portfolio(
    stocks = dict(zip(stock_symbols, stocks)),
    capital=500000,
    variations=2
    )

backtester = Backtester(
    portfolio = CC_RSI_portfolio,
    variations = CC_RSI_portfolio.variations
    )

In [163]:
backtester.prepare_strategy()

In [164]:
backtester.run()

Day 0
Day 0
Day 0
Day 0
Day 0
Day 0
Day 0
Day 0
Day 0
Day 0
Bought 25408 shares of NVDA - Total Capital Left : 400000.3123626709
Bought 14829 shares of AMZN - Total Capital Left : 300000.947412014
Bought 13710 shares of AAPL - Total Capital Left : 200005.8733305931
Bought 2402 shares of PG - Total Capital Left : 100027.69041585922
Bought 7090 shares of GOOGL - Total Capital Left : 40.23543882369995
Bought 25408 shares of NVDA - Total Capital Left : 400000.3123626709
Bought 14829 shares of AMZN - Total Capital Left : 300000.947412014
Bought 13710 shares of AAPL - Total Capital Left : 200005.8733305931
Bought 2402 shares of PG - Total Capital Left : 100027.69041585922
Bought 7090 shares of GOOGL - Total Capital Left : 40.23543882369995
Bought 16112 shares of NVDA - Total Capital Left : 436676.6731454212
Bought 17702 shares of AMZN - Total Capital Left : 311912.9796439487
Bought 17152 shares of AAPL - Total Capital Left : 187151.3652640659
Bought 1497 shares of PG - Total Capital Left : 1

In [166]:
print(backtester.portfolio.dataframe.index[0])
print(backtester.portfolio.dataframe.index[-1])

2010-04-07
2016-12-30


In [167]:
backtester.portfolio.add_benchmark('SPY')

[*********************100%%**********************]  1 of 1 completed
(1698, 6)
(1698, 2)
Match


In [168]:
backtester.portfolio.dataframe

,Portfolio_1_value,Portfolio_2_value,benchmark_SPY_close
2010-04-07,5.000000e+05,5.000000e+05,118.360001
2010-04-08,4.990549e+05,4.990549e+05,118.769997
2010-04-09,4.994741e+05,4.994741e+05,119.550003
2010-04-12,4.995127e+05,4.994064e+05,119.739998
2010-04-13,4.996612e+05,4.995549e+05,119.830002
...,...,...,...
2016-12-23,1.672204e+06,1.684787e+06,225.710007
2016-12-27,1.680751e+06,1.693397e+06,226.270004
2016-12-28,1.690759e+06,1.703480e+06,224.399994
2016-12-29,1.674267e+06,1.686864e+06,224.350006


In [93]:
backtester.portfolio.dataframe

,Portfolio_1_value,Portfolio_2_value,Portfolio_3_value
2010-06-03,5.000000e+05,5.000000e+05,5.000000e+05
2010-06-04,4.910704e+05,4.910704e+05,4.910704e+05
2010-06-07,4.948934e+05,4.948934e+05,4.948934e+05
2010-06-08,4.959350e+05,4.959350e+05,4.959350e+05
2010-06-09,4.993510e+05,4.993510e+05,4.993510e+05
...,...,...,...
2016-12-23,1.714189e+06,1.727949e+06,1.721301e+06
2016-12-27,1.722950e+06,1.736780e+06,1.730098e+06
2016-12-28,1.733209e+06,1.747121e+06,1.740399e+06
2016-12-29,1.716302e+06,1.730079e+06,1.723423e+06


In [169]:
# One shot run code

stock_symbols = ['NVDA','EMR','PG','GOOGL','MMM']

stocks = [ Stock(
    symbol=symbol,
    start='2017-01-01',
    end='2019-12-31', 
    interval='1d', 
    transcation_cost=0, 
    strategy=CC_RSI_strategy, 
    verbose=True) 
    for symbol in stock_symbols
    ]

CC_RSI_portfolio = Portfolio(
    stocks = dict(zip(stock_symbols, stocks)),
    capital=500000,
    variations=1
    )

backtester_test = Backtester(
    portfolio = CC_RSI_portfolio,
    variations = CC_RSI_portfolio.variations
    )

backtester_test.prepare_strategy()
backtester_test.run()
backtester_test.portfolio.add_benchmark('SPY')

Day 0
Day 0
Day 0
Day 0
Day 0
Bought 3406 shares of NVDA - Total Capital Left : 400009.5500831604
Bought 1918 shares of EMR - Total Capital Left : 300009.69192123413
Bought 1365 shares of PG - Total Capital Left : 200077.44956588745
Bought 2411 shares of GOOGL - Total Capital Left : 100113.77725982666
Bought 719 shares of MMM - Total Capital Left : 203.3646011352539
Bought 3412 shares of NVDA - Total Capital Left : 398939.71299939696
Bought 1916 shares of EMR - Total Capital Left : 299251.7775288159
Bought 1357 shares of PG - Total Capital Left : 199544.00974469725
Bought 2403 shares of GOOGL - Total Capital Left : 99833.92899137083
Bought 712 shares of MMM - Total Capital Left : 158.8070431286469
Bought 3481 shares of NVDA - Total Capital Left : 399443.22961603594
Bought 1910 shares of EMR - Total Capital Left : 299589.7871569295
Bought 1353 shares of PG - Total Capital Left : 199793.29910456133
Bought 2405 shares of GOOGL - Total Capital Left : 99978.5880678792
Bought 708 shares of M

In [170]:
# Sample run
backtester.check_unique_signal_count()
backtester.verify()
backtester.portfolio.stocks['PG'].data.cc_1_weights.value_counts()

NVDA
cc_1_val
2    1040
1     627
3      31
Name: count, dtype: int64
cc_2_val
2    1049
1     633
3      16
Name: count, dtype: int64


AMZN
cc_1_val
2    1114
1     549
3      35
Name: count, dtype: int64
cc_2_val
2    1135
1     549
3      14
Name: count, dtype: int64


AAPL
cc_1_val
2    1089
1     576
3      33
Name: count, dtype: int64
cc_2_val
2    1097
1     585
3      16
Name: count, dtype: int64


PG
cc_1_val
2    1057
1     612
3      29
Name: count, dtype: int64
cc_2_val
2    1055
1     623
3      20
Name: count, dtype: int64


GOOGL
cc_1_val
2    1036
1     628
3      34
Name: count, dtype: int64
cc_2_val
2    1046
1     634
3      18
Name: count, dtype: int64




cc_1_weights
0.200000    414
0.222222    318
0.250000    199
0.125000    157
0.111111    125
0.142857    125
0.166667    121
0.285714    111
0.333333     47
0.181818     40
0.100000     13
0.272727     12
0.300000     10
0.375000      4
0.428571      1
0.083333      1
Name: count, dtype: int64